In [18]:
import autograd.numpy as jnp  # Thinly-wrapped numpy
from autograd import grad, jacobian    # The only autograd function you may ever need

In [19]:
import drawSvg as draw
from drawSvg.widgets import DrawingWidget
import hyperbolic.poincare.shapes as hyper  # pip3 install hyperbolicd
import numpy as np


canvas_size=(800,800)
# Create drawing
d = draw.Drawing(*canvas_size, origin='center')

linkage = draw.Group()
d.append(linkage)
traj=draw.Group()
d.append(traj)


def redraw_linkage(nodes=[], edges=[],node_radius=6,edge_width=5):
    linkage.children.clear()
    for id1,id2 in edges:
        linkage.draw(draw.Line(*nodes[id1],*nodes[id2],stroke='gray',stroke_width=edge_width))
    for x,y in nodes[1:-1]:
        linkage.draw(draw.Circle(x,y,node_radius,fill='green'))
    x,y=nodes[0]
    linkage.draw(draw.Circle(x,y,node_radius,fill='blue'))
    x,y=nodes[-1]
    linkage.draw(draw.Circle(x,y,node_radius,fill='orange'))

def redraw_trajectory(drive_node=[],end_node=[]):
    traj.children.clear()
    for p0,p1 in zip(drive_node[:-1],drive_node[1:]):
        traj.draw(draw.Line(*p0,*p1,stroke='blue',stroke_width=3))
    for p0,p1 in zip(end_node[:-1],end_node[1:]):
        traj.draw(draw.Line(*p0,*p1,stroke='orange',stroke_width=3))
        
        
def rotation_matrix(theta=0):
    c, s = jnp.cos(theta), jnp.sin(theta)
    R = jnp.array(((c, -s), (s, c)))
    return R

def simulate_linkage(nodes=[],edges=[],lengths=[]):
    for i in range(0,len(edges),2):
        e02,e12 = edges[[i,i+1],:]
        l02,l12 = lengths[[i,i+1]]
        p0, p1 = nodes[[e02[0],e12[0]],:]
        
        if np.linalg.norm(p1-p0) > l02 + l12:
            p0 = p1 - (l02+l12)*(p1-p0) / np.linalg.norm(p1 - p0)
            nodes[e02[0],:] = p0
        
        p2 = simulate_linkage_impl(p0,p1,l12,l02)
        print(p2)
        print(jacobian(simulate_linkage_impl)(p0,p1,l12,l02))
        nodes[e02[1],:] = p2
        
def simulate_linkage_impl(p1, p0, l12, l02):
    v01 = (p1-p0) / jnp.linalg.norm(p1 - p0)
    l01 = jnp.linalg.norm(p1 - p0)
    phi = jnp.arccos(max(-1,min(1,(l01**2+l02**2-l12**2)/(2*l01*l02))))
    return jnp.dot(rotation_matrix(phi), v01*l02+p0)


In [20]:
nodes=np.array([(100,100),(0,0),(0,0),(0,0),(0,0),(0,0),(0,0)])
edges=np.array([(0,2),(1,2),(2,3),(1,3),(1,4),(0,4),(3,5),(4,5),(5,6),(4,6)])
lengths=np.array([100,100,100,100,100,150,100,100,150,100])
drive_node=[]
end_node=[]
# Create interactive widget and register mouse events
widget = DrawingWidget(d)
@widget.mousedown
def mousedown(widget, x, y, info):
    drive_node.append([*nodes[0]])
    if len(drive_node)>1:
        for i in range(0,10):
            nodes[0,:]=(1-i/10.0)*np.asarray(drive_node[-2])+(i/10.0)*np.asarray(drive_node[-1])
            simulate_linkage(nodes,edges,lengths)
            end_node.append([*nodes[-1]])
        nodes[0,:]=drive_node[-1]
    else:
        end_node.append([*nodes[-1]])
    redraw_trajectory(drive_node,end_node)
    redraw_linkage(nodes,edges)
    widget.refresh()
@widget.mousemove
def mousemove(widget, x, y, info):
    nodes[0]=(x,y)
    simulate_linkage(nodes,edges,lengths)
    redraw_trajectory(drive_node,end_node)
    redraw_linkage(nodes,edges)
    widget.refresh()

widget

DrawingWidget()

[97.54127037 22.03861555]
[[  5100439.66312687   1152400.48037175]
 [-22574165.92569112  -5100438.66312687]]
[29.31055466 95.608009  ]
[[ 0.7499762  -0.81555188]
 [-0.22992026  0.2500238 ]]
[50.95459606 85.93098333]
[[-0.66234661 -0.59055446]
 [ 0.39275244  0.35018177]]
[-131.81000466  -25.12744216]
[[-1.45091996 -3.34115306]
 [ 0.38429877  2.35041133]]
[-73.63340818 -27.22699947]
[[ 0.43770331 -0.42719301]
 [-0.55327644  0.11791897]]
[-96.09411193 -27.67528955]
[[-0.18762434 -0.3420381 ]
 [ 0.65146906  1.18762434]]
[-24.53126177 -96.9444026 ]
[[ 0.80150842 -0.78441328]
 [-0.20281741  0.19849158]]
[10.12552593 20.83207994]
[[-0.68944188 -1.01192239]
 [ 0.33510632  0.49184942]]
[ 49.56213874 -60.35345357]
[[0.74487539 0.17639045]
 [0.55945871 0.16016073]]
[131.89886471 -39.03892678]
[[0.66500853 0.19428661]
 [1.55790837 0.32057665]]
[-96.85138517  24.89596735]
[[0.03099046 0.12056045]
 [0.12056045 0.46900954]]
[-69.07689005 -72.30756019]
[[ 0.22638218 -0.8097993 ]
 [-0.21626752  0.77361

 [ 0.15657266  1.00776402]]
[-99.97014733   2.44328508]
[[0.00547389 0.02430636]
 [0.2239711  0.99452611]]
[-51.25486502 -85.86581864]
[[ 0.47652767 -0.87695832]
 [-0.28444801  0.52347233]]
[-23.26856548 -13.34285128]
[[-0.38293883 -0.14680411]
 [ 0.66780608  0.25601133]]
[ 75.07398259 -95.66797837]
[[1.02560729 0.15757533]
 [1.01573381 0.04163654]]
[141.23307072 -21.49590771]
[[-0.03682478 -0.00429599]
 [ 0.62468909  1.00751051]]
[-99.89572582   4.56551889]
[[0.01567085 0.04498664]
 [0.34288558 0.98432915]]
[-53.00688724 -84.79545922]
[[ 0.45313207 -0.87482816]
 [-0.28325951  0.54686793]]
[-16.95764652  -9.70982085]
[[-0.39540507 -0.15738679]
 [ 0.69055232  0.27486703]]
[ 64.44883246 -95.98622893]
[[ 1.06684428  0.06282408]
 [ 0.83213998 -0.0149553 ]]
[143.40020731 -31.32069302]
[[0.07694908 0.09170356]
 [0.88505303 0.91543739]]
[-99.93231367   3.67867964]
[[0.01397584 0.03629724]
 [0.3796574  0.98602416]]
[-52.13140873 -85.33648823]
[[ 0.4648155  -0.87607005]
 [-0.28395224  0.5351845

[-71.91345127  69.48708892]
[[ 1.52283309 -0.50519268]
 [ 1.57600764 -0.52283309]]
[-96.05511544 -27.81033618]
[[-0.08100022 -0.31297635]
 [ 0.27976956  1.08100022]]
[28.78026794 47.5500079 ]
[[-1.64769402  0.99728849]
 [ 0.99728849 -0.60362198]]
[-38.84613325 -43.10553184]
[[ 0.60964859 -0.4136995 ]
 [-0.3352648   0.01398725]]
[  4.06050036 -95.59679451]
[[ 1.07772413 -0.79242483]
 [ 0.10489814 -0.07701424]]
[-93.97616902  34.18303169]
[[ 1.06386411 -0.02323003]
 [ 2.92478076 -0.06386411]]
[-76.33270539 -64.60122357]
[[ 0.12515394 -0.74039202]
 [-0.1478817   0.87484606]]
[12.42885351 66.28721869]
[[-2.96716132  0.55634275]
 [ 0.55634275 -0.10431427]]
[-24.164949   -40.49260203]
[[ 0.70927466 -0.08553183]
 [-0.07075566 -0.1875866 ]]
[ 23.67834865 -80.82934871]
[[ 1.1677772  -0.39551371]
 [ 0.49688996 -0.16843581]]
[-96.86661069  24.83666107]
[[0.83219133 0.04302625]
 [3.24566789 0.16780867]]
[-69.07689005 -72.30756019]
[[ 0.22638218 -0.8097993 ]
 [-0.21626752  0.77361782]]
[ 6.9503719 

[56.17136069 82.73317495]
[[-0.46855682 -2.16299493]
 [ 0.31812479  1.46855682]]
[-43.68314039  89.95433978]
[[ 1.04031233 -0.08301301]
 [ 0.50519085 -0.04031233]]
[ 56.914503   -24.39192986]
[[-0.40750272 -0.95083969]
 [-0.95083969 -2.21862593]]
[-41.24337473  31.93849942]
[[-0.09433025  0.23898417]
 [ 0.29055448  0.66988614]]
[-88.84109444 -12.39847845]
[[-0.14113933 -0.24825132]
 [ 0.65043141  1.14234411]]
[95.98310217 28.05787051]
[[-0.96958138 -0.57574988]
 [ 3.31683862  1.96958138]]
[22.93852651 97.33357078]
[[ 0.8151634  -0.78430522]
 [-0.19210892  0.1848366 ]]
[ 29.87261114 -68.70700563]
[[-3.35335633 -1.45798101]
 [-1.45798101 -0.63390479]]
[ 2.44767949 40.31614624]
[[ 0.46949012  0.37762154]
 [ 0.36302006 -0.00631611]]
[-57.18135414  52.86138286]
[[0.73443538 0.18125658]
 [1.0823389  0.26804023]]
[99.70366114 -7.69285096]
[[0.26134018 0.05699289]
 [3.38711522 0.73865982]]
[55.6234135  83.10256236]
[[ 0.4183483  -0.869     ]
 [-0.28001496  0.5816517 ]]
[  4.88020916 -70.763032

...

[ 63.9964683  -76.84043235]
[[ 1.00704912 -0.00846387]
 [ 0.83871974 -0.00704912]]
[98.45627024 17.50322401]
[[-0.07228557 -0.19062732]
 [ 0.40660894  1.07228557]]
[-15.80200783   1.80546816]
[[ 0.13329205 -0.03393088]
 [ 1.16661265 -0.29697343]]
[35.60643893 77.56190099]
[[ 0.57728966 -0.7086078 ]
 [-0.26991577  0.35989844]]
[-30.56668598 139.88934404]
[[ 1.16524122 -1.20593073]
 [ 0.1258063  -0.17876245]]
[ 70.96755524 -70.45286441]
[[0.97497648 0.02484203]
 [0.98209914 0.02502352]]
[96.44102864 26.44102864]
[[-0.08127747 -0.29645151]
 [ 0.29645151  1.08127747]]
[-21.43630629  -0.78438723]
[[-0.04805134  0.00513648]
 [ 1.31318201 -0.14037356]]
[28.54902586 74.33685136]
[[ 0.61820634 -0.64166638]
 [-0.24851363  0.29634487]]
[-33.19305612 135.50279614]
[[ 1.17257742 -1.270386  ]
 [ 0.10514777 -0.19062383]]
[ 80.97001166 -58.68438644]
[[0.83362799 0.12058093]
 [1.15020148 0.16637201]]
[91.0324338  41.38956386]
[[-0.05315157 -0.47883466]
 [ 0.11690186  1.05315157]]
[-24.15449184  -5.6809

[84.21249339 53.92824823]
[[ 0.0236968  -0.62520796]
 [-0.0370041   0.9763032 ]]
[-24.50795765 -10.24453209]
[[-0.54320049 -0.13583023]
 [ 1.29949659  0.32494618]]
[ 7.47416278 73.93413363]
[[ 0.80774795 -0.44471788]
 [-0.10048984  0.07724237]]
[-62.15635761 118.24986123]
[[ 1.36748731 -0.89818744]
 [ 0.4641816  -0.37702092]]
[ 89.11014017 -45.3804244 ]
[[0.63926553 0.18370843]
 [1.25527783 0.36073447]]
[83.61194664 54.8547389 ]
[[ 0.03419012 -0.63363252]
 [-0.05211405  0.96580988]]
[-24.43994768 -10.66392346]
[[-0.56219777 -0.14977888]
 [ 1.28846425  0.34326841]]
[ 6.22109353 74.10335589]
[[ 0.81468543 -0.43890824]
 [-0.0891596   0.07156432]]
[-64.40091129 117.39537824]
[[ 1.36693551 -0.85631969]
 [ 0.49034104 -0.37707018]]
[ 90.34964157 -42.85956449]
[[0.6001103  0.18969747]
 [1.26505603 0.3998897 ]]
[81.70587375 57.65544375]
[[ 0.05547172 -0.66650284]
 [-0.07861123  0.94452828]]
[-24.25336311 -11.47030401]
[[-0.59790426 -0.17871611]
 [ 1.26423756  0.37788595]]
[ 3.65861365 73.883865

[99.77520134 -6.70143243]
[[0.08160822 0.06168407]
 [1.21503531 0.91839178]]
[54.75323932 83.67844874]
[[ 0.42988916 -0.87129074]
 [-0.28128896  0.57011084]]
[-19.4967629  -22.90413394]
[[-0.90466473 -0.7767515 ]
 [ 0.7700808   0.66119679]]
[-21.86247928  67.71391013]
[[ 0.87023109 -0.06870058]
 [ 0.25212554 -0.002129  ]]
[-99.55193894  83.45139205]
[[ 1.26920968 -0.22078742]
 [ 1.12066981 -0.27222538]]
[99.79079428 -6.46508911]
[[0.08433247 0.05932283]
 [1.30169967 0.91566753]]
[54.75323932 83.67844874]
[[ 0.42988916 -0.87129074]
 [-0.28128896  0.57011084]]
[-20.11015167 -26.81353556]
[[-0.98341024 -0.9478747 ]
 [ 0.73755768  0.71090603]]
[-18.86083596  64.58561625]
[[ 0.84872415 -0.05742231]
 [ 0.22895781 -0.00394198]]
[-92.85678813  83.79881674]
[[ 1.2893159  -0.26308542]
 [ 1.04267971 -0.28294502]]
[99.7935113  -6.42301343]
[[0.08993143 0.05857478]
 [1.39725269 0.91006857]]
[54.75323932 83.67844874]
[[ 0.42988916 -0.87129074]
 [-0.28128896  0.57011084]]
[-20.0036385  -31.05096498]


[ 39.64380565 -91.80614726]
[[ 1.39458941 -0.91378043]
 [ 0.60221274 -0.39458941]]
[ 99.36254355 -11.27319562]
[[0.07910394 0.10448043]
 [0.69722634 0.92089606]]
[-42.44607636  -2.34621393]
[[-0.13330305  0.01333815]
 [ 2.41162642 -0.24130465]]
[46.75385673 34.99452309]
[[ 0.21689485 -0.48288036]
 [-0.29579718  0.55085626]]
[ 37.20373903 104.85614167]
[[ 0.34434858 -1.38071038]
 [-0.20222603  0.68556065]]
[ 28.94459801 -95.71943505]
[[ 1.28100348 -0.92927512]
 [ 0.38736262 -0.28100348]]
[ 97.33357078 -22.93852651]
[[0.1848366  0.19210892]
 [0.78430522 0.8151634 ]]
[-35.21675051   8.29268203]
[[ 0.39124266 -0.16239777]
 [ 1.66150048 -0.68965887]]
[52.73619596 36.4330877 ]
[[ 0.14834171 -0.46389047]
 [-0.21452261  0.67234896]]
[ 46.94549351 111.01931721]
[[ 0.15856729 -1.41632136]
 [-0.15010566  0.85696486]]
[ 23.92875626 -97.09487434]
[[ 1.22743462 -0.92285346]
 [ 0.30249778 -0.22743462]]
[ 95.85312524 -28.4987435 ]
[[0.24024008 0.22588938]
 [0.80802729 0.75975992]]
[-30.89920526  11.24

[[ 0.08557158 -0.48651167]
 [-0.10174453  0.82338325]]
[ 53.96737753 127.20427221]
[[ 0.13505554 -1.49503958]
 [-0.13446676  0.85035346]]
[ 14.93087743 -98.87906199]
[[ 1.11326984 -0.7501244 ]
 [ 0.16810531 -0.11326984]]
[ 93.01744009 -36.71179427]
[[0.33724231 0.26157486]
 [0.85447788 0.66275769]]
[-15.87306293  11.40414858]
[[ 0.53995535 -0.51661696]
 [ 0.75154626 -0.71906232]]
[68.52593669 44.9612115 ]
[[ 0.08360698 -0.50495854]
 [-0.09652693  0.84061663]]
[ 52.22209137 130.98266734]
[[ 0.19047871 -1.51764748]
 [-0.15061402  0.79288728]]
[ 14.5596234  -98.93440942]
[[ 1.10711314 -0.72784683]
 [ 0.16292765 -0.10711314]]
[ 93.01744009 -36.71179427]
[[0.33724231 0.26157486]
 [0.85447788 0.66275769]]
[-14.56238951  10.85802753]
[[ 0.53306015 -0.52638341]
 [ 0.71492078 -0.70596618]]
[69.32533808 46.12728003]
[[ 0.08537254 -0.51973985]
 [-0.10141827  0.84367123]]
[ 49.09036478 133.14172923]
[[ 0.26040694 -1.5085531 ]
 [-0.16907867  0.7246696 ]]
[ 14.86946473 -98.88831589]
[[ 1.1033924  -0

In [ ]:
result

In [ ]:
# dynamics simulation 

def get_force(angle, mass): return mass * -9.81 * np.sin(angle)
# game loop implementation

# system parameters
pendulum_mass = 0.7  #kg
pendulum_string_length = 0.1 #m

# step
dt = 0.01

# system state (incl. initial conditions)
pendulum_angle = 1.2
pendulum_angular_velocity = 0

result = []

# step between 0s and 10s every dt seconds
for step in np.arange(0.0, 100.0, dt):
    # get acceleration
    # F = m*a  ->  a = F/m
    
    # TODO implement update call
    
    result.append(pendulum_angle)
    
plt.plot(result)
plt.show()

In [ ]:
# dynamics simulation 

def get_force(angle, mass): return mass * -9.81 * np.sin(angle)
# game loop implementation

# system parameters
pendulum_mass = 0.7  #kg
pendulum_string_length = 0.1 #m

# step
dt = 0.01

# system state (incl. initial conditions)
pendulum_angle = 1.2
pendulum_angular_velocity = 0

result = []

# step between 0s and 10s every dt seconds
for step in np.arange(0.0, 100.0, dt):
    # get acceleration
    # F = m*a  ->  a = F/m
    acceleration = get_force(pendulum_angle, pendulum_mass) / pendulum_mass
    
    # do the time step
    pendulum_angle += pendulum_angular_velocity * dt
    pendulum_angular_velocity += acceleration * dt
    result.append(pendulum_angle)
    
plt.plot(result)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# is an approximation since time is discretized

# let's be a little smarter about discretization

# battle plan formulate system as a continuous-time model -> pass it to a differential equation solver

# function that returns dy/dt
# variable naming convention from control theory:
# y -> system state vector (sometimes 'u')
# t -> time
# p -> parameter vector
def model(t, y, p=()):
    pendulum_angle = y[0]
    pendulum_anglular_velocity = y[1]
    
    # TODO implement model

# initial condition
y0 = [0,10]

from scipy.integrate import solve_ivp

# time points
time_span = (0.0, 100.0)

# solve ODEs
k = 0.1
result = solve_ivp(model, time_span, y0, method='RK45', rtol=1e-8)

fig = plt.figure()
ax = fig.add_subplot(1, 2, 2)
ax.plot(result.y[0, :],
        result.y[1, :])
ax.set_title("solve_ivp")
result
# time plot

In [ ]:
# @Lukas TODO put it on the repo in seperate file
import matplotlib.pyplot as plt

# is an approximation since time is discretized

# let's be a little smarter about discretization

# battle plan formulate system as a continuous-time model -> pass it to a differential equation solver

# function that returns dy/dt
# variable naming convention from control theory:
# y -> system state vector (sometimes 'u')
# t -> time
# p -> parameter vector
def model(t, y, p=()):
    pendulum_angle = y[0]
    pendulum_anglular_velocity = y[1]
    
    acceleration = get_force(pendulum_angle, pendulum_mass) / pendulum_mass
    
    return [pendulum_angular_velocity, acceleration]

# initial condition
y0 = [0,10]

from scipy.integrate import solve_ivp


# time points
time_span = (0.0, 100.0)

# solve ODEs
k = 0.1
result = solve_ivp(model, time_span, y0, method='RK45', rtol=1e-8)

fig = plt.figure()
ax = fig.add_subplot(1, 2, 2)
ax.plot(result.y[0, :],
        result.y[1, :])
ax.set_title("solve_ivp")
result
# time plot

In [ ]:
# TODO state-space-plot


In [ ]:
# TODO make this work for the linkage system


In [ ]:
# TODO run FFT on the data
